# Assistant Service

In [5]:
import json
import sys
import os
sys.path.append(os.path.join(os.getcwd(),'..','..'))
import ibm_watson
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [6]:
API_KEY = os.environ.get('ASSISTANT_APIKEY','<ASSISTANT_APIKEY>')

In [7]:
authenticator = IAMAuthenticator(API_KEY)
assistant = ibm_watson.AssistantV1(version='2018-07-10', authenticator=authenticator)

## Pizza Chatbot

Let's create and chat with a simple pizza bot. We'll start by defining the bot's workspace, then add intents and examples to recognize pizza orders. Once the chatbot is configured, we'll send a message to converse with our pizza bot.

In [8]:
# define the dialog for our example workspace
dialog_nodes = [{'conditions': '#order_pizza',
                 'context': None,
                 'description': None,
                 'dialog_node': 'YesYouCan',
                 'go_to': None,
                 'metadata': None,
                 'output': {'text': {'selection_policy': 'random',
                                     'values': ['Yes You can!', 'Of course!']}},
                 'parent': None,
                 'previous_sibling': None,}]

# create an example workspace
workspace = assistant.create_workspace(name='example_workspace',
                                          description='An example workspace.',
                                          language='en',
                                          dialog_nodes=dialog_nodes).get_result()

# print response and save workspace_id
print(json.dumps(workspace, indent=2))
workspace_id=workspace['workspace_id']

{
  "learning_opt_out": false, 
  "workspace_id": "8db6b689-c4a7-4bb0-a2a7-86e58587f8f9", 
  "language": "en", 
  "name": "example_workspace", 
  "metadata": {
    "api_version": {
      "major_version": "v1", 
      "minor_version": "2018-07-10"
    }
  }, 
  "description": "An example workspace."
}


In [9]:
# add an intent to the workspace
intent = assistant.create_intent(workspace_id=workspace_id,
                                    intent='order_pizza',
                                    description='A pizza order.').get_result()
print(json.dumps(intent, indent=2))

{
  "intent": "order_pizza", 
  "description": "A pizza order."
}


In [10]:
# add examples to the intent
example1 = assistant.create_example(workspace_id=workspace_id,
                                       intent='order_pizza',
                                       text='Can I order a pizza?').get_result()
example2 = assistant.create_example(workspace_id=workspace_id,
                                       intent='order_pizza',
                                       text='I want to order a pizza.').get_result()
example3 = assistant.create_example(workspace_id=workspace_id,
                                       intent='order_pizza',
                                       text='pizza order').get_result()
example4 = assistant.create_example(workspace_id=workspace_id,
                                       intent='order_pizza',
                                       text='pizza to go').get_result()

print(json.dumps(example1, indent=2))
print(json.dumps(example2, indent=2))
print(json.dumps(example3, indent=2))
print(json.dumps(example4, indent=2))

{
  "text": "Can I order a pizza?"
}
{
  "text": "I want to order a pizza."
}
{
  "text": "pizza order"
}
{
  "text": "pizza to go"
}


In [13]:
# check workspace status (wait for training to complete)
workspace = assistant.get_workspace(workspace_id=workspace_id).get_result()
print('The workspace status is: {0}'.format(workspace['status']))
if workspace['status'] == 'Available':
    print('Ready to chat!')
else:
    print('The workspace should be available shortly. Please try again in 30s.')
    print('(You can send messages, but not all functionality will be supported yet.)')

The workspace status is: Available
Ready to chat!


In [14]:
# start a chat with the pizza bot
input = {'text': 'Can I order a pizza?'}
response = assistant.message(workspace_id=workspace_id,
                                input=input).get_result()
print(json.dumps(response, indent=2))

{
  "entities": [], 
  "intents": [
    {
      "confidence": 1, 
      "intent": "order_pizza"
    }
  ], 
  "output": {
    "generic": [
      {
        "text": "Of course!", 
        "response_type": "text"
      }
    ], 
    "text": [
      "Of course!"
    ], 
    "log_messages": [], 
    "nodes_visited": [
      "YesYouCan"
    ]
  }, 
  "context": {
    "conversation_id": "f673c936-c8e6-41d1-9bae-771ef92e2f3d", 
    "system": {
      "dialog_stack": [
        {
          "dialog_node": "root"
        }
      ], 
      "dialog_request_counter": 1, 
      "dialog_turn_counter": 1, 
      "branch_exited": true, 
      "initialized": true, 
      "_node_output_map": {
        "YesYouCan": [
          0, 
          1, 
          0
        ]
      }, 
      "branch_exited_reason": "completed"
    }
  }, 
  "input": {
    "text": "Can I order a pizza?"
  }
}


In [15]:
# continue a chat with the pizza bot
# (when you send multiple requests for the same conversation,
#  then include the context object from the previous response)
input = {'text': 'medium'}
response = assistant.message(workspace_id=workspace_id,
                             input=input,
                             context=response['context']).get_result()
print(json.dumps(response, indent=2))

{
  "entities": [], 
  "intents": [], 
  "output": {
    "generic": [], 
    "text": [], 
    "warning": "No dialog node condition matched to true in the last dialog round - context.nodes_visited is empty. Falling back to the root node in the next round.", 
    "log_messages": [
      {
        "msg": "No dialog node condition matched to true in the last dialog round - context.nodes_visited is empty. Falling back to the root node in the next round.", 
        "level": "warn"
      }
    ], 
    "nodes_visited": []
  }, 
  "context": {
    "conversation_id": "f673c936-c8e6-41d1-9bae-771ef92e2f3d", 
    "system": {
      "dialog_stack": [
        {
          "dialog_node": "root"
        }
      ], 
      "dialog_request_counter": 2, 
      "dialog_turn_counter": 2, 
      "branch_exited": true, 
      "initialized": true, 
      "_node_output_map": {
        "YesYouCan": [
          0, 
          1, 
          0
        ]
      }, 
      "branch_exited_reason": "fallback"
    }
  }, 
  

## Operation Examples

The following examples demonstrate each operation of the Assistant service. They use the pizza chatbot workspace configured above.

### Message

In [16]:
input = {'text': 'Can I order a pizza?'}
response = assistant.message(workspace_id=workspace_id,
                                input=input).get_result()
print(json.dumps(response, indent=2))

{
  "entities": [], 
  "intents": [
    {
      "confidence": 1, 
      "intent": "order_pizza"
    }
  ], 
  "output": {
    "generic": [
      {
        "text": "Of course!", 
        "response_type": "text"
      }
    ], 
    "text": [
      "Of course!"
    ], 
    "log_messages": [], 
    "nodes_visited": [
      "YesYouCan"
    ]
  }, 
  "context": {
    "conversation_id": "f354f7ca-0996-47e6-b0aa-cc85f877eab1", 
    "system": {
      "dialog_stack": [
        {
          "dialog_node": "root"
        }
      ], 
      "dialog_request_counter": 1, 
      "dialog_turn_counter": 1, 
      "branch_exited": true, 
      "initialized": true, 
      "_node_output_map": {
        "YesYouCan": [
          0, 
          1, 
          0
        ]
      }, 
      "branch_exited_reason": "completed"
    }
  }, 
  "input": {
    "text": "Can I order a pizza?"
  }
}


### Workspaces

In [17]:
response = assistant.create_workspace(name='test_workspace',
                                         description='Test workspace.',
                                         language='en',
                                         metadata={}).get_result()
print(json.dumps(response, indent=2))
test_workspace_id = response['workspace_id']

{
  "learning_opt_out": false, 
  "workspace_id": "a98ceedd-1d76-4e55-86f2-0fdbf79233d4", 
  "language": "en", 
  "name": "test_workspace", 
  "metadata": {
    "api_version": {
      "major_version": "v1", 
      "minor_version": "2018-07-10"
    }
  }, 
  "description": "Test workspace."
}


In [18]:
response = assistant.delete_workspace(workspace_id=test_workspace_id).get_result()
print(json.dumps(response, indent=2))

{}


In [19]:
response = assistant.get_workspace(workspace_id=workspace_id, export=True).get_result()
print(json.dumps(response, indent=2))

{
  "status": "Available", 
  "learning_opt_out": false, 
  "intents": [
    {
      "intent": "order_pizza", 
      "examples": [
        {
          "text": "Can I order a pizza?"
        }, 
        {
          "text": "I want to order a pizza."
        }, 
        {
          "text": "pizza order"
        }, 
        {
          "text": "pizza to go"
        }
      ], 
      "description": "A pizza order."
    }
  ], 
  "description": "An example workspace.", 
  "language": "en", 
  "dialog_nodes": [
    {
      "output": {
        "text": {
          "values": [
            "Yes You can!", 
            "Of course!"
          ], 
          "selection_policy": "random"
        }
      }, 
      "conditions": "#order_pizza", 
      "type": "standard", 
      "dialog_node": "YesYouCan", 
      "title": "YesYouCan"
    }
  ], 
  "workspace_id": "8db6b689-c4a7-4bb0-a2a7-86e58587f8f9", 
  "entities": [], 
  "counterexamples": [], 
  "metadata": {
    "api_version": {
      "major_versio

In [20]:
response = assistant.list_workspaces().get_result()
print(json.dumps(response, indent=2))

{
  "pagination": {
    "refresh_url": "/v1/workspaces?version=2018-07-10"
  }, 
  "workspaces": [
    {
      "learning_opt_out": false, 
      "workspace_id": "8db6b689-c4a7-4bb0-a2a7-86e58587f8f9", 
      "language": "en", 
      "name": "example_workspace", 
      "metadata": {
        "api_version": {
          "major_version": "v1", 
          "minor_version": "2018-07-10"
        }
      }, 
      "description": "An example workspace."
    }
  ]
}


In [21]:
response = assistant.update_workspace(workspace_id=workspace_id,
                                         description='An example workspace for ordering pizza.').get_result()
print(json.dumps(response, indent=2))

{
  "learning_opt_out": false, 
  "workspace_id": "8db6b689-c4a7-4bb0-a2a7-86e58587f8f9", 
  "language": "en", 
  "name": "example_workspace", 
  "metadata": {
    "api_version": {
      "major_version": "v1", 
      "minor_version": "2018-07-10"
    }
  }, 
  "description": "An example workspace for ordering pizza."
}


### Intents

In [22]:
response = assistant.create_intent(workspace_id=workspace_id,
                                      intent='test_intent',
                                      description='Test intent.').get_result()
print(json.dumps(response, indent=2))

{
  "intent": "test_intent", 
  "description": "Test intent."
}


In [23]:
response = assistant.delete_intent(workspace_id=workspace_id,
                                      intent='test_intent').get_result()
print(json.dumps(response, indent=2))

{}


In [24]:
response = assistant.get_intent(workspace_id=workspace_id,
                                   intent='order_pizza',
                                   export=True).get_result()
print(json.dumps(response, indent=2))

{
  "intent": "order_pizza", 
  "examples": [
    {
      "text": "Can I order a pizza?"
    }, 
    {
      "text": "I want to order a pizza."
    }, 
    {
      "text": "pizza order"
    }, 
    {
      "text": "pizza to go"
    }
  ], 
  "description": "A pizza order."
}


In [25]:
response = assistant.list_intents(workspace_id=workspace_id,
                                     export=True).get_result()
print(json.dumps(response, indent=2))

{
  "intents": [
    {
      "intent": "order_pizza", 
      "examples": [
        {
          "text": "Can I order a pizza?"
        }, 
        {
          "text": "I want to order a pizza."
        }, 
        {
          "text": "pizza order"
        }, 
        {
          "text": "pizza to go"
        }
      ], 
      "description": "A pizza order."
    }
  ], 
  "pagination": {
    "refresh_url": "/v1/workspaces/8db6b689-c4a7-4bb0-a2a7-86e58587f8f9/intents?version=2018-07-10&export=true"
  }
}


In [26]:
response = assistant.update_intent(workspace_id=workspace_id,
                                      intent='order_pizza',
                                      new_intent='order_pizza',
                                      new_description='Order a pizza.').get_result()
print(json.dumps(response, indent=2))

{
  "intent": "order_pizza", 
  "description": "Order a pizza."
}


### Examples

In [27]:
response = assistant.create_example(workspace_id=workspace_id,
                                       intent='order_pizza',
                                       text='Gimme a pizza with pepperoni').get_result()
print(json.dumps(response, indent=2))

{
  "text": "Gimme a pizza with pepperoni"
}


In [28]:
response = assistant.delete_example(workspace_id=workspace_id,
                                       intent='order_pizza',
                                       text='Gimme a pizza with pepperoni').get_result()
print(json.dumps(response, indent=2))

{}


In [29]:
response = assistant.get_example(workspace_id=workspace_id,
                                    intent='order_pizza',
                                    text='I want to order a pizza.').get_result()
print(json.dumps(response, indent=2))

{
  "text": "I want to order a pizza."
}


In [30]:
response = assistant.list_examples(workspace_id=workspace_id,
                                      intent='order_pizza').get_result()
print(json.dumps(response, indent=2))

{
  "pagination": {
    "refresh_url": "/v1/workspaces/8db6b689-c4a7-4bb0-a2a7-86e58587f8f9/intents/order_pizza/examples?version=2018-07-10"
  }, 
  "examples": [
    {
      "text": "Can I order a pizza?"
    }, 
    {
      "text": "I want to order a pizza."
    }, 
    {
      "text": "pizza order"
    }, 
    {
      "text": "pizza to go"
    }
  ]
}


In [31]:
response = assistant.update_example(workspace_id=workspace_id,
                                       intent='order_pizza',
                                       text='I want to order a pizza.',
                                       new_text='I want to order a pizza with pepperoni.').get_result()
print(json.dumps(response, indent=2))

{
  "text": "I want to order a pizza with pepperoni."
}


### Counterexamples

In [32]:
response = assistant.create_counterexample(workspace_id=workspace_id,
                                              text='I want financial advice today.').get_result()
print(json.dumps(response, indent=2))

{
  "text": "I want financial advice today."
}


In [33]:
response = assistant.get_counterexample(workspace_id=workspace_id,
                                           text='I want financial advice today.').get_result()
print(json.dumps(response, indent=2))

{
  "text": "I want financial advice today."
}


In [34]:
response = assistant.list_counterexamples(workspace_id=workspace_id).get_result()
print(json.dumps(response, indent=2))

{
  "pagination": {
    "refresh_url": "/v1/workspaces/8db6b689-c4a7-4bb0-a2a7-86e58587f8f9/counterexamples?version=2018-07-10"
  }, 
  "counterexamples": [
    {
      "text": "I want financial advice today."
    }
  ]
}


In [35]:
response = assistant.update_counterexample(workspace_id=workspace_id,
                                              text='I want financial advice today.',
                                              new_text='I want financial advice for tomorrow.').get_result()
print(json.dumps(response, indent=2))

{
  "text": "I want financial advice for tomorrow."
}


In [36]:
response = assistant.delete_counterexample(workspace_id=workspace_id,
                                              text='I want financial advice for tomorrow.').get_result()
print(json.dumps(response, indent=2))

{}


## Entities

In [37]:
values = [{"value": "juice"}]

In [38]:
response = assistant.create_entity(workspace_id=workspace_id,
                                      entity='test_entity',
                                      description='A test entity.',
                                      values=values).get_result()
print(json.dumps(response, indent=2))

{
  "description": "A test entity.", 
  "entity": "test_entity"
}


In [39]:
response = assistant.get_entity(workspace_id=workspace_id,
                                   entity='test_entity',
                                   export=True).get_result()
print(json.dumps(response, indent=2))

{
  "values": [
    {
      "synonyms": [], 
      "type": "synonyms", 
      "value": "juice"
    }
  ], 
  "description": "A test entity.", 
  "entity": "test_entity"
}


In [40]:
response = assistant.list_entities(workspace_id=workspace_id).get_result()
print(json.dumps(response, indent=2))

{
  "entities": [
    {
      "description": "A test entity.", 
      "entity": "test_entity"
    }
  ], 
  "pagination": {
    "refresh_url": "/v1/workspaces/8db6b689-c4a7-4bb0-a2a7-86e58587f8f9/entities?version=2018-07-10"
  }
}


In [41]:
response = assistant.update_entity(workspace_id=workspace_id,
                                      entity='test_entity',
                                      new_description='An updated test entity.').get_result()
print(json.dumps(response, indent=2))

{
  "description": "An updated test entity.", 
  "entity": "test_entity"
}


In [42]:
response = assistant.delete_entity(workspace_id=workspace_id,
                                      entity='test_entity').get_result()
print(json.dumps(response, indent=2))

{}


## Synonyms

In [43]:
values = [{"value": "orange juice"}]
assistant.create_entity(workspace_id, 'beverage', values=values).get_result()

{u'entity': u'beverage'}

In [44]:
response = assistant.create_synonym(workspace_id, 'beverage', 'orange juice', 'oj').get_result()
print(json.dumps(response, indent=2))

{
  "synonym": "oj"
}


In [45]:
response = assistant.get_synonym(workspace_id, 'beverage', 'orange juice', 'oj').get_result()
print(json.dumps(response, indent=2))

{
  "synonym": "oj"
}


In [46]:
response = assistant.list_synonyms(workspace_id, 'beverage', 'orange juice').get_result()
print(json.dumps(response, indent=2))

{
  "synonyms": [
    {
      "synonym": "oj"
    }
  ], 
  "pagination": {
    "refresh_url": "/v1/workspaces/8db6b689-c4a7-4bb0-a2a7-86e58587f8f9/entities/beverage/values/orange%20juice/synonyms?version=2018-07-10"
  }
}


In [47]:
response = assistant.update_synonym(workspace_id, 'beverage', 'orange juice', 'oj', 'OJ').get_result()
print(json.dumps(response, indent=2))

{
  "synonym": "OJ"
}


In [48]:
response = assistant.delete_synonym(workspace_id, 'beverage', 'orange juice', 'OJ').get_result()
print(json.dumps(response, indent=2))

{}


In [49]:
assistant.delete_entity(workspace_id, 'beverage').get_result()

{}

## Values

In [50]:
assistant.create_entity(workspace_id, 'test_entity').get_result()

{u'entity': u'test_entity'}

In [51]:
response = assistant.create_value(workspace_id, 'test_entity', 'test').get_result()
print(json.dumps(response, indent=2))

{
  "type": "synonyms", 
  "value": "test"
}


In [52]:
response = assistant.get_value(workspace_id, 'test_entity', 'test').get_result()
print(json.dumps(response, indent=2))

{
  "type": "synonyms", 
  "value": "test"
}


In [53]:
response = assistant.list_values(workspace_id, 'test_entity').get_result()
print(json.dumps(response, indent=2))

{
  "pagination": {
    "refresh_url": "/v1/workspaces/8db6b689-c4a7-4bb0-a2a7-86e58587f8f9/entities/test_entity/values?version=2018-07-10"
  }, 
  "values": [
    {
      "type": "synonyms", 
      "value": "test"
    }
  ]
}


In [54]:
response = assistant.update_value(workspace_id, 'test_entity', 'test', 'example').get_result()
print(json.dumps(response, indent=2))

{
  "type": "synonyms", 
  "value": "example"
}


In [55]:
response = assistant.delete_value(workspace_id, 'test_entity', 'example').get_result()
print(json.dumps(response, indent=2))

{}


In [56]:
assistant.delete_entity(workspace_id, 'test_entity').get_result()

{}

## Logs

In [57]:
response = assistant.list_logs(workspace_id=workspace_id).get_result()
print(json.dumps(response, indent=2))

{
  "pagination": {}, 
  "logs": [
    {
      "workspace_id": "8db6b689-c4a7-4bb0-a2a7-86e58587f8f9", 
      "language": "en", 
      "request_timestamp": "2019-03-18T18:45:04.031Z", 
      "log_id": "c5619c32-28bc-4518-b7af-ae5ea6297aac", 
      "request": {
        "input": {
          "text": "Can I order a pizza?"
        }
      }, 
      "response_timestamp": "2019-03-18T18:45:04.051Z", 
      "response": {
        "entities": [], 
        "intents": [
          {
            "confidence": 1, 
            "intent": "order_pizza"
          }
        ], 
        "output": {
          "generic": [
            {
              "text": "Of course!", 
              "response_type": "text"
            }
          ], 
          "text": [
            "Of course!"
          ], 
          "log_messages": [], 
          "nodes_visited": [
            "YesYouCan"
          ]
        }, 
        "context": {
          "conversation_id": "f673c936-c8e6-41d1-9bae-771ef92e2f3d", 
          "syste

## Cleanup (Delete Pizza Chatbot)

Let's cleanup by deleting the pizza chatbot, since it is no longer needed.

In [58]:
# clean-up by deleting the workspace
assistant.delete_workspace(workspace_id=workspace_id).get_result()

{}